## Experimental interactive sketch

#### Imports

In [158]:
from __future__ import print_function # For py 2.7 compat
import ipywidgets as widgets
import IPython.display 
from IPython.display import Javascript, display
from traitlets import Unicode
from traitlets import CBytes
from traitlets import CInt
from traitlets import CBool
import traitlets
import numpy as np
from PIL import Image, ImageDraw
import numpy as np
from ipywidgets import interact, interactive


#### Globals

In [2]:
p55_globals = {
    "canvas_width": 320,
    "canvas_height": 320
}

In [3]:
def update_javascript_globals():
    ##### Copy globals to javascript
    javascript_code = "var p55_globals = {\n"

    num_items = len(p55_globals)
    for i, k in enumerate(p55_globals):
        javascript_code += "  \"{}\": {}".format(k, p55_globals[k])
        # the dreded comma
        if i < num_items - 1:
            javascript_code += ","
        javascript_code += "\n"
    javascript_code += "};\n"
    javascript_code += "window.p55_globals = p55_globals\n"
    return javascript_code

In [4]:
##### Copy globals to javascript
js_code = update_javascript_globals()
print(js_code)
# strangely, this cannot be called from within a python function (WTF)
Javascript(js_code)

var p55_globals = {
  "canvas_width": 320,
  "canvas_height": 320
};
window.p55_globals = p55_globals



<IPython.core.display.Javascript object>

In [5]:
#### p55 drawing convenience methods
class P55DrawContext():
    drawContext = None
    cur_fill = (0, 0, 0, 255)
    cur_outline = (0, 0, 0, 255)
    
    def fill(self, c):
        self.cur_fill = (c, c, c, 255)
    
    def ellipse(self, x1, y1, rx, ry):
        if self.drawContext is None:
            return
        hrx = int(rx / 2)
        hry = int(ry / 2)
        self.drawContext.ellipse((x1-hrx, y1-hry, x1+hrx, y1+hry), self.cur_fill, self.cur_outline)

    def rectangle(self, x1, y1, x2, y2):
        if self.drawContext is None:
            return
        self.drawContext.rectangle([x1, y1, x2, y2], self.cur_fill, self.cur_outline)

    def line(self, x1, y1, x2, y2):
        if self.drawContext is None:
            return
        self.drawContext.line((x1, y1, x2, y2), self.cur_fill)


#### Python side canvas widget

In [6]:
class P55CanvasWidget(widgets.DOMWidget, P55DrawContext):
    _view_name = Unicode('P55CanvasView').tag(sync=True)
    _view_module = Unicode('p55canvas').tag(sync=True)
    value = CInt().tag(sync=True)
    temp_array = np.zeros([p55_globals["canvas_width"],p55_globals["canvas_height"],4], dtype='uint8') + 128
    image_buffer = CBytes(temp_array.tobytes()).tag(sync=True)
    mouseX = CInt().tag(sync=True)
    mouseY = CInt().tag(sync=True)
    mousePressed = CBool().tag(sync=True)    

#### Javascript side

In [7]:
%%javascript
requirejs.undef('p55canvas');

var p55_globals = window.p55_globals;

define('p55canvas', ["jupyter-js-widgets"], function(widgets) {
    // we track all mouse movement and store into globals
    var p55_mouse_x = 0;
    var p55_mouse_y = 0;
    var p55_mouse_is_down = false;

    var callback_on_mouse_down = function(evt) {
        p55_mouse_is_down = true;
    };
    
    var callback_on_mouse_up = function(evt) {
        p55_mouse_is_down = false;
    };

    function track_mouse_pos(evt) {
        p55_mouse_x = evt.clientX;
        p55_mouse_y = evt.clientY;
    }

    var P55CanvasView = widgets.DOMWidgetView.extend({
        render: function() { 
            var that = this;
            var canvas_str = '<canvas width="' + p55_globals["canvas_width"] + '" height="' + p55_globals["canvas_height"] + '"></canvas>'
            this.$canvas = $(canvas_str);
            this.$el.append(this.$canvas);
            this.value_changed();
            this.model.on('change:value', this.value_changed, this);
            this.$canvas[0].addEventListener('mousemove', track_mouse_pos, false);
            this.$canvas[0].addEventListener('mousedown', callback_on_mouse_down, false);
            this.$canvas[0].addEventListener('mouseup', callback_on_mouse_up, false);
        },
        
        value_changed: function() {
            // anytime the value is changed we do a full sync
            var ctx = this.$canvas[0].getContext("2d");
            var buffer = this.model.get('image_buffer').buffer
            var arr = new Uint8ClampedArray(buffer);
            var imdata = new ImageData(arr, p55_globals["canvas_width"], p55_globals["canvas_height"]);
            ctx.putImageData(imdata, 0, 0)
            var rect = this.$canvas[0].getBoundingClientRect();
            var local_x = Math.floor( p55_mouse_x - rect.left );
            var local_y = Math.floor( p55_mouse_y - rect.top );
            this.handle_mouse(local_x, local_y, p55_mouse_is_down);
        },
        
        handle_mouse: function(x, y, is_down) {
            this.model.set('mouseX', x);
            this.model.set('mouseY', y);
            this.model.set('mousePressed', is_down);
            this.touch();
        },
        
    });
    
    return {
        P55CanvasView: P55CanvasView
    };
});

<IPython.core.display.Javascript object>

### Try it out

In [175]:
def get_p55_play_observer(wrapper):
    def play_observer(change):
        backend_img = wrapper.backend_img
        p55_widget = wrapper.p55_widget
        value = change['new']
        if value == 0:
            wrapper.reset_widget();
            backend_img = wrapper.backend_img
        else:
            wrapper.step_func(p55_widget)
            if value % wrapper.redrawFreq == 0:
                wrapper.render_func(p55_widget)
            wrapper.draw_func(p55_widget)
        p55_widget.image_buffer = backend_img.tobytes()
    return play_observer

def get_label_changer(wrapper):
    def on_value_change(change):
        wrapper.step_label.value = u"{}".format(change['new'])
    return on_value_change

def empty_function(wrapper):
    pass

class P55Wrapper():
    def reset_widget(self):
        color = (0,0,0,255)
        self.backend_img = Image.new('RGBA', [p55_globals["canvas_width"], p55_globals["canvas_height"]], color=color)
        self.p55_widget.drawContext = ImageDraw.Draw(self.backend_img)
        self.setup_func(self.p55_widget)
        
    def __init__(self, setupfn=empty_function, drawfn=empty_function, stepfn=empty_function, renderfn=empty_function):
        self.setup_func = setupfn
        self.draw_func = drawfn
        self.step_func = stepfn
        self.render_func = renderfn
        self.redrawFreq = 1

        self.play = widgets.Play()
        self.p55_widget = P55CanvasWidget()
        self.p55_widget.value = 0
        self.reset_widget();
        self.step_label = widgets.Label(u"0", description='step');

        play_observer = get_p55_play_observer(self)
        self.p55_widget.observe(play_observer, names='value')
        traitlets.link((self.play, 'value'), (self.p55_widget, 'value'))
        label_observer = get_label_changer(self)
        self.p55_widget.observe(label_observer, names='value')
        
#         traitlets.link((self.play, 'value'), (self.step_label, 'value'))

        self.play_row = widgets.HBox([self.play, self.step_label])
        self.display_pane = widgets.VBox([self.p55_widget, self.play_row])
        self.settings_pane = None
        self.radius = 120
        self.widget = widgets.HBox([self.display_pane])
    
    def addSettingsPane(self, settings_pane):
        self.settings_pane = settings_pane
        self.widget = widgets.HBox([self.display_pane, settings_pane])   
        

In [157]:
def draw(self):
    if self.mousePressed:
        self.fill(0)
    else:
        self.fill(255)
    self.ellipse(self.mouseX, self.mouseY, 80, 80)

wrapper = P55Wrapper(drawfn=draw)
display(wrapper.widget)

In [156]:
def wrapped_widgets(wrapper):
    def settings_updated(radius, color):
        if radius < 0:
            radius = 0
        wrapper.p55_widget.radius = radius
        t = webcolors.html5_parse_legacy_color(color)
        wrapper.p55_widget.color = (t[0], t[1], t[2], 255)
        return radius, color
    return settings_updated

def draw(self):
    self.fill(128)
    if self.mousePressed:
        self.fill(0)
    else:
        self.cur_fill = self.color
    self.ellipse(self.mouseX, self.mouseY, self.radius, self.radius)

wrapper = P55Wrapper(drawfn=draw)
settings_callback = wrapped_widgets(wrapper)
cp = widgets.ColorPicker()
cp.value = "white"
w = interactive(settings_callback, radius=80, color=cp)
wrapper.addSettingsPane(w)
wrapper.play.max = 1000
display(wrapper.widget)


(89, '#445dff')

In [10]:
def draw1(self):
    if not self.mousePressed:
        color = (255,0,0,255)
    else:
        color = (0,0,255,255)
    self.cur_fill = color
    self.line(0, 0, self.mouseX, self.mouseY)

wrapper = P55Wrapper(draw1)
display(wrapper.widget)    

In [185]:
BACK = (255,255,255,255)
FRONT = (0, 0, 0, 255)
LIGHT = (0, 100, 0, 255)
CYAN = (0, 100, 100, 255)
BLUE = (0, 0, 200, 255)
RED = (128, 0, 0, 255)

FRONTCOLORS = [FRONT, LIGHT, CYAN, BLUE, RED]

In [186]:
NMAX = 10**6
SIZE = 200
ONE = 1.5/SIZE
LINEWIDTH = ONE*1.1

INIT_NUM = 4

STP = ONE*3

ANGLE_STP = 0.5
ANGLE_LOCAL_STP = 0.8

In [202]:
from modules.wind import Wind

def setup_wind(self):
    global ANGLE_STP, ANGLE_LOCAL_STP
    self.wind = Wind(NMAX, SIZE, STP, ANGLE_STP, ANGLE_LOCAL_STP)
    self.wind.rnd_seed(INIT_NUM)

def step_wind(self):
    res = self.wind.step()

def render_wind(self):
    self.fill(225)
    self.cur_outline = self.cur_fill
    self.rectangle(0,0,320,320)
    self.fill(0)

    xy = self.wind.xy
    n = self.wind.n
    r = self.wind.r

    for i,x in enumerate(xy[:n,:]):
        self.cur_fill = FRONTCOLORS[r[i]]
        self.cur_outline = FRONTCOLORS[r[i]]
        self.ellipse(int(320*x[0]), int(320*x[1]), 2, 2)

    for i,p in enumerate(self.wind.p[:n]):
        if p>-1:
            self.cur_fill = FRONTCOLORS[r[i]]
            self.cur_outline = FRONTCOLORS[r[i]]
            source = 320 * xy[p,:].flatten()
            dest = 320 * xy[i,:].flatten()
            self.line(source[0], source[1], dest[0], dest[1])


In [189]:
wrapper = P55Wrapper(setupfn=setup_wind, stepfn=step_wind, renderfn=render_wind)
wrapper.play.max = 1000
wrapper.redrawFreq = 5
display(wrapper.widget)

In [209]:
def wrapped_wind_settings(wrapper):
    def settings_updated(drawFreq, ASTP, ALOCAL, color):
        global FRONTCOLORS
        global ANGLE_STP
        global ANGLE_LOCAL_STP
        wrapper.redrawFreq = drawFreq
        ANGLE_STP = ASTP
        ANGLE_LOCAL_STP = ALOCAL
        t = webcolors.html5_parse_legacy_color(color)
        FRONTCOLORS[0] = (t[0], t[1], t[2], 255)
        return None
    return settings_updated

In [210]:
wrapper = P55Wrapper(setupfn=setup_wind, stepfn=step_wind, renderfn=render_wind)
settings_callback = wrapped_wind_settings(wrapper)
cp = widgets.ColorPicker()
rw = widgets.IntSlider(value=1,min=1,max=20)
asw = widgets.FloatSlider(value=0.5,min=0,max=2)
alsw = widgets.FloatSlider(value=0.8,min=0,max=2)
w = interactive(settings_callback, drawFreq=rw, ASTP=asw, ALOCAL=alsw, color=cp)
wrapper.addSettingsPane(w)
wrapper.play.max = 1000
wrapper.redrawFreq = 5
display(wrapper.widget)
